In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Code due to Hugo Carrillo at Inria-Chile 2022
def spectral_burgers(D, T_max=1, file_name='u_burgers'):
    N = 256
    NT = 100
    h = 2 * np.pi / N
    XX = np.arange(N) * h
    XXshift = np.roll(XX, N // 2)
    XXshift2 = XXshift[0::2]
    X = (1 / np.pi) * np.angle(np.exp(1j * XXshift2))
    X[0] = -1

    X = np.delete(X, N // 4)

    xx = np.append(X, 1)

    dt = T_max * 2e-6

    # Create the wave number vector k
    k_positive = np.arange(0, N // 2)
    k_negative = np.arange(-N // 2 + 1, 0)
    k = np.concatenate((k_positive, [0], k_negative)) * np.pi
    k1 = 1j * k
    k2 = k1 ** 2

    uinit = -np.sin(XX)
    u0 = uinit.copy()

    T = np.linspace(0, T_max, NT)
    uburgers = np.zeros((N // 2, NT))

    cont = 0
    cont_t = -1
    TT = np.arange(0, T_max, dt)
    for t in TT:
        cont_t += 1
        u1hat = np.fft.fft(u0.copy()) - dt * k1 * np.fft.fft(0.5 * u0.copy() ** 2) + dt * D * k2 * np.fft.fft(u0.copy())
        u0 = np.fft.ifft(u1hat.copy()).real

        if np.abs(TT[cont_t] - T[cont]) < 1e-2:
            uburgers[:, cont] = np.roll(u0.copy()[1::2], N // 4)
            cont += 1
            if cont > 99:
                break

    uburgers = np.delete(uburgers, N // 4, axis=0)
    u_burgers = np.vstack((uburgers, uburgers[0, :]))
    x = xx
    t = T

    return (x, t, u_burgers)

In [ ]:
D_test = np.array([0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1])

dic = {}

print("Creating solution for D={}".format(D_test[0]))
x, t, u_burgers = spectral_burgers(D_test[0]/np.pi)
dic["D_"+str(D_test[0])] = u_burgers
print("Solution created for D={}".format(D_test[0]))

for D in D_test[1:]:
    print("Creating solution for D={}".format(D))
    _, _, u_burgers = spectral_burgers(D/np.pi)
    dic["D_"+str(D)] = u_burgers
    print("Solution created for D={}".format(D))

Creating solution for D=0.0025
Solution created for D=0.0025
Creating solution for D=0.005
Solution created for D=0.005
Creating solution for D=0.0075
Solution created for D=0.0075
Creating solution for D=0.01
Solution created for D=0.01
Creating solution for D=0.025
Solution created for D=0.025
Creating solution for D=0.05
Solution created for D=0.05
Creating solution for D=0.075
Solution created for D=0.075
Creating solution for D=0.1
Solution created for D=0.1


In [ ]:
np.savez("burgers_data.npz", x=x, t=t, **dic)